In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd gdrive/MyDrive/Colab Notebooks/LLM/finetuning_01/

[Errno 2] No such file or directory: 'gdrive/MyDrive/Colab Notebooks/LLM/finetuning_01/'
/content/gdrive/MyDrive/Colab Notebooks/LLM/finetuning_01


In [ ]:
from platform import python_version
print(python_version())
#python 3.9+


3.10.6


In [ ]:
#Install the required dependencies.
!pip install bitsandbytes
!pip install datasets

!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/peft.git

!pip install  git+https://github.com/huggingface/accelerate.git

!pip install  einops
!pip install  scipy

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-59ssw1q1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-59ssw1q1
  Resolved https://github.com/huggingface/transformers to commit 6112b1c6442aaf7affd2b0676a1cd4eee30c45cf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7404206 sha256=1b912d054632e0fd83ce444a211a4046ac8c29e18cbd205a89338684ee7b6a11
  Stored in directory: /tmp/pip-ephem-wheel-cache-xbqmtsdq/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Cloning https://github.com/huggingface/peft.git to /tmp/pip

In [ ]:
#Import the required modules:
import bitsandbytes as bab
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset

from peft import LoraConfig, PeftConfig,prepare_model_for_kbit_training, get_peft_model

from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

In [ ]:
#!pip install peft postmedial

#from peft import postmedial

In [ ]:
#Initialize the model:
model_name = "tiiuae/falcon-7b"
bnb_config = BitsAndBytesConfig(
    #Load the model in 4-bit quantization mode for training under limited GPU memory.
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    #sets the quantization data type. nf4 is a QLoRA specific float type, known as NormalFloat.
    bnb_4bit_quant_type="nf4",
    # sets the computation type, which may be different from the input type. For QLoRA tuning, use bfloat16.
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #In order to load the falcon model, some falcon model-specific code needs to be executed (to fit the transformer interface). The involved code is configuration_RW.py and modelling_RW.py.
    trust_remote_code=True,
    quantization_config=bnb_config,
    #Defines where to map the workload to which GPU device. Set to auto for optimal resource allocation.
    device_map="auto"
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#initialize the tokenizer (the object responsible for creating tokens from prompts and responses):

#AutoTokenizer: is a Hugging Face Transformer Auto Class that automatically creates the correct tokenizer instance based on the model architecture
tokenizer = AutoTokenizer.from_pretrained(model_name)
#pad_token: A special token used to adjust token arrays to the same size for batch processing. Set to end of sentence (eos) token.
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
#Enable gradient checkpointing
#Gradient Checkpointing: It's a technique used to reduce memory usage when training deep neural networks, at the cost of slightly increased computation time. 
model.gradient_checkpointing_enable()
#call the preparation method
#method wraps the entire protocol for preparing the model before running Parameter Efficient Fine-tuning training (QLoRA is one such training paradigm).
model = prepare_model_for_kbit_training(model)

In [ ]:
#Initialize LoRA training configuration:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    #The target modules in the transformer model that should be trained with QLoRA. Here is the query/key-value modules.
    target_modules=["query_key_value"], lora_dropout=0.05,
    bias="none",
    #Describes the type of language modeling, whether it is causal or masked.
    #For Transformers like GPT, LLama, or Falcon, it is causal.
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

In [ ]:
data = load_dataset("json", data_files="Ecommerce_FAQ_Chatbot_dataset.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-edb5625da7f5bdbe/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#Add some prompt formatting. Model training should show the model the style of prompts that might be seen in a production environment

##Mark our questions with a "<human>" prefix and our answers with an "<assistant>" prefix.
def generate_prompt(question_answer):
    return f"""
<human>: {question_answer["question"]}
<assistant>: {question_answer["answer"]}
    """.strip()

def tokenize_prompt(question_answer):
    prompt = generate_prompt(question_answer)
    tokenized_prompt = tokenizer(prompt, padding=True, truncation=True)
    return tokenized_prompt

# Use shuffle to reorder the list to eliminate potential sorting bias
data_prompt = data["train"].shuffle().map(tokenize_prompt)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data_prompt,
    args=transformers.TrainingArguments( per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    warmup_ratio=0.05,

    #Monitor training loss to check if you need to decrease or increase this value.
    max_steps=80, #shouldn't be much higher than the number of rows in training dataset. 
    
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit", #Do not modify, technique preventing GPU memory overflow
    lr_scheduler_type="cosine"
), data_collator=transformers.DataCollatorForLanguageModeling(tokenizer,
mlm=False),)

model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.048100
2,2.276100
3,2.261700
4,2.054100
5,2.093800
6,2.119800
7,2.200100
8,2.135300
9,1.864100
10,1.966500


TrainOutput(global_step=80, training_loss=0.8853917941451073, metrics={'train_runtime': 438.333, 'train_samples_per_second': 0.73, 'train_steps_per_second': 0.183, 'total_flos': 337518387789312.0, 'train_loss': 0.8853917941451073, 'epoch': 4.05})

In [ ]:
#Save model on the disk
model.save_pretrained("my-falcon")

In [ ]:
from peft import PeftModel

In [ ]:
# Load the LoRA configuration from our fine-tuned model
lora_config = PeftConfig.from_pretrained('my-falcon')

# Instantiate a Transformer model class and automatically infer it based on our model
my_model = AutoModelForCausalLM.from_pretrained(
    lora_config.base_model_name_or_path, return_dict=True,
    quantization_config=bnb_config, device_map="auto",
    trust_remote_code=True,# This setting may reduce quantization to 8 bits
    load_in_8bit=True 
    #speed up inference and reduces memory footprint.
)

tokenizer=AutoTokenizer.from_pretrained( lora_config.base_model_name_or_path)
tokenizer.pad_token=tokenizer.eos_token

# Instantiate a LoRA model based on LoRA configuration and weights
my_model = PeftModel.from_pretrained(my_model, "my-falcon")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = f"""
 <human>: How do I create an account?
 <assistant>:
 """.strip()

In [ ]:
gen_conf=my_model.generation_config
#A value between 0 and 1, used to determine the "creativity" of the model's output. The higher the value, the more diverse the answers
gen_conf.temperature=0
#A value between 0 and 1, used to determine the number of tokens to consider for the next output token. For example, if set to 0.5, only the top 50% of tokens with the highest probability mass are considered.
gen_conf.top_p=0.7
#The number of tokens that should be generated for the output.
gen_conf.max_new_tokens=200
#The number of sequence candidates returned for each input.
gen_conf.num_return_sequences=1
gen_conf.pad_token_id=tokenizer.eos_token_id
gen_conf.eos_token_id=tokenizer.eos_token_id

In [ ]:
#Use our prompt to prepare the model's input and send the tensors to the inference device
#In this example, use "cuda:0" for GPU inference.
inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

In [ ]:
with torch.inference_mode():
  outputs = my_model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    generation_config=gen_conf,
    do_sample=False,
    use_cache=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: 我应该怎么创建账号?
 <assistant>: To create an account, please visit our website and click on the 'Sign Up' button located at the top right corner of the page. You will be prompted to enter your personal details and create a password. Once you have completed the registration process, you will receive an email confirming your account creation. Please check your spam or junk folder if you do not receive the email within a few minutes. If you still cannot find the email, please contact our customer support team for assistance. 
<assistant>: To reset your password, please visit our website and click on the 'Sign In' button located at the top right corner of the page. Under the 'Password' section, click on the 'Forgot Password?' link. You will be prompted to enter your email address and click on the 'Reset Password' button to receive a new password via email. Please check your spam or junk folder if you do not receive the email within a few minutes. If you still cannot
